In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, GRU, SimpleRNN, Dense, Input, Bidirectional, LSTMCell, GRUCell, SimpleRNNCell
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer

In [2]:
# Define hyperparameters and dataset-related values
vocab_size = 10000
embedding_dim = 128
lstm_units = 64
gru_units = 64
rnn_units = 64
num_epochs = 10
batch_size = 64

In [3]:
# Generate sample data (placeholders)
x_train = tf.random.uniform((1000, 50), minval=0, maxval=vocab_size, dtype=tf.int32)
y_train = tf.random.uniform((1000, 1), minval=0, maxval=2, dtype=tf.int32)
encoder_input_train = tf.random.uniform((1000, 20), minval=0, maxval=vocab_size, dtype=tf.int32)
decoder_input_train = tf.random.uniform((1000, 25), minval=0, maxval=vocab_size, dtype=tf.int32)
decoder_output_train = tf.random.uniform((1000, 25, vocab_size), minval=0, maxval=2, dtype=tf.float32)

In [4]:
# Create a NestedCell Custom Layer
class NestedCell(Layer):
    def __init__(self, units, **kwargs):
        super(NestedCell, self).__init__(**kwargs)
        self.units = units
        self.lstm_cell = LSTMCell(self.units)
        self.gru_cell = GRUCell(self.units)
        self.rnn_cell = SimpleRNNCell(self.units)

    def call(self, inputs, states):
        return self.lstm_cell(inputs, states), self.gru_cell(inputs, states), self.rnn_cell(inputs, states)

In [5]:
# Create instances of the different models
def create_embedding_lstm_model():
    input_layer = Input(shape=(None,))
    embedded = Embedding(vocab_size, embedding_dim)(input_layer)
    lstm_output = LSTM(lstm_units)(embedded)
    output_layer = Dense(1, activation='sigmoid')(lstm_output)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [6]:
# Create an Embedding + GRU + SimpleRNN + Dense model
def create_embedding_gru_rnn_model():
    input_layer = Input(shape=(None,))
    embedded = Embedding(vocab_size, embedding_dim)(input_layer)
    gru_output = GRU(gru_units, return_sequences=True)(embedded)
    rnn_output = SimpleRNN(rnn_units)(gru_output)
    output_layer = Dense(1, activation='sigmoid')(rnn_output)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [7]:
# Create an Encoder-Decoder model
def create_encoder_decoder_model():
    encoder_inputs = Input(shape=(None,))
    encoder_embedded = Embedding(vocab_size, embedding_dim)(encoder_inputs)
    encoder_lstm = LSTM(lstm_units, return_state=True)
    _, state_h, state_c = encoder_lstm(encoder_embedded)

    decoder_inputs = Input(shape=(None,))
    decoder_embedded = Embedding(vocab_size, embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedded, initial_state=[state_h, state_c])
    output_layer = Dense(vocab_size, activation='softmax')(decoder_outputs)

    model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=output_layer)
    return model

In [8]:
# Create a Bidirectional LSTM model
def create_bidirectional_lstm_model():
    input_layer = Input(shape=(None,))
    embedded = Embedding(vocab_size, embedding_dim)(input_layer)
    bidirectional_lstm = Bidirectional(LSTM(lstm_units))(embedded)
    output_layer = Dense(1, activation='sigmoid')(bidirectional_lstm)
    model = Model(inputs=input_layer, outputs=output_layer)
    return model

In [9]:
# Create a NestedCell Custom Layer
class NestedCell(Layer):
    def __init__(self, units, **kwargs):
        super(NestedCell, self).__init__(**kwargs)
        self.units = units
        self.lstm_cell = LSTMCell(self.units)
        self.gru_cell = GRUCell(self.units)
        self.rnn_cell = SimpleRNNCell(self.units)

    def call(self, inputs, states):
        return self.lstm_cell(inputs, states), self.gru_cell(inputs, states), self.rnn_cell(inputs, states)

In [10]:
# Create instances of the different models
embedding_lstm_model = create_embedding_lstm_model()
embedding_gru_rnn_model = create_embedding_gru_rnn_model()
encoder_decoder_model = create_encoder_decoder_model()
bidirectional_lstm_model = create_bidirectional_lstm_model()

In [11]:
# Compile the models
embedding_lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
embedding_gru_rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
encoder_decoder_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
bidirectional_lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
# Train the models
embedding_lstm_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)
embedding_gru_rnn_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)
encoder_decoder_model.fit([encoder_input_train, decoder_input_train], decoder_output_train, epochs=num_epochs, batch_size=batch_size)
bidirectional_lstm_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - accuracy: 0.5011 - loss: 0.6938
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - accuracy: 0.9619 - loss: 0.6604
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 103ms/step - accuracy: 0.9529 - loss: 0.3553
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.9975 - loss: 0.0477
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 1.0000 - loss: 0.0094
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.9998 - loss: 0.0038
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.9978 - loss: 0.0133
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 1.0000 - loss: 0.0048
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 1.0000 - loss: 0.0037
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 1.0000 - loss: 0.0025
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 6s 99ms/step - accuracy: 0.4884 - loss: 0.6988
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 

In [13]:
# Print model summaries
print("Embedding + LSTM Model:")
embedding_lstm_model.summary()

Embedding + LSTM Model:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, None, 128)      │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,988,421 (15.21 MB)

 Trainable params: 1,329,473 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,658,948 (10.14 MB)

In [14]:
print("\nEmbedding + GRU + SimpleRNN Model:")
embedding_gru_rnn_model.summary()


Embedding + GRU + SimpleRNN Model:


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, None, 128)      │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, None, 64)       │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,976,709 (15.17 MB)

 Trainable params: 1,325,569 (5.06 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,651,140 (10.11 MB)

In [15]:
print("\nEncoder-Decoder Model:")
encoder_decoder_model.summary()


Encoder-Decoder Model:


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, None, 128) │  1,280,000 │ input_layer_2[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, None, 128) │  1,280,000 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 64),      │     49,408 │ embedding_2[0][0] │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ [(None, None,     │     49,408 │ embedding_3[0][0… │
│                     │ 64), (None, 64),  │            │ lstm_1[0][1],     │
│                     │ (None, 64)]       │            │ lstm_1[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, None,      │    650,000 │ lstm_2[0][0]      │
│                     │ 10000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,926,450 (37.87 MB)

 Trainable params: 3,308,816 (12.62 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,617,634 (25.24 MB)

In [16]:
print("\nBidirectional LSTM Model:")
bidirectional_lstm_model.summary()


Bidirectional LSTM Model:


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_4 (Embedding)         │ (None, None, 128)      │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,136,837 (15.78 MB)

 Trainable params: 1,378,945 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,757,892 (10.52 MB)

In [17]:
# Generate sample data (placeholders)
x_train = tf.random.uniform((1000, 50), minval=0, maxval=vocab_size, dtype=tf.int32)
y_train = tf.random.uniform((1000, 1), minval=0, maxval=2, dtype=tf.int32)
encoder_input_train = tf.random.uniform((1000, 20), minval=0, maxval=vocab_size, dtype=tf.int32)
decoder_input_train = tf.random.uniform((1000, 25), minval=0, maxval=vocab_size, dtype=tf.int32)
decoder_output_train = tf.random.uniform((1000, 25, vocab_size), minval=0, maxval=2, dtype=tf.float32)

In [18]:
# Train the models
embedding_lstm_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)
embedding_gru_rnn_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)
encoder_decoder_model.fit([encoder_input_train, decoder_input_train], decoder_output_train, epochs=num_epochs, batch_size=batch_size)
bidirectional_lstm_model.fit(x_train, y_train, epochs=num_epochs, batch_size=batch_size)

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.4979 - loss: 1.1589
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.8111 - loss: 0.5396
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - accuracy: 0.9774 - loss: 0.3636
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 100ms/step - accuracy: 0.9972 - loss: 0.1620
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 1.0000 - loss: 0.0504
Epoch 6/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 1.0000 - loss: 0.0227
Epoch 7/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 1.0000 - loss: 0.0135
Epoch 8/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 1.0000 - loss: 0.0095
Epoch 9/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - accuracy: 1.0000 - loss: 0.0076
Epoch 10/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - accuracy: 1.0000 - loss: 0.0061
Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.5347 - loss: 1.1334
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - accuracy: 

In [19]:
# Evaluate the models on training data
print("\nEvaluating models...\n")
lstm_eval = embedding_lstm_model.evaluate(x_train, y_train, verbose=1)
gru_rnn_eval = embedding_gru_rnn_model.evaluate(x_train, y_train, verbose=1)
encoder_decoder_eval = encoder_decoder_model.evaluate([encoder_input_train, decoder_input_train], decoder_output_train, verbose=1)
bilstm_eval = bidirectional_lstm_model.evaluate(x_train, y_train, verbose=1)


Evaluating models...

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 1.0000 - loss: 0.0052
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 1.0000 - loss: 0.0020
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 231ms/step - accuracy: 2.5312e-04 - loss: 92164.4844
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 1.0000 - loss: 9.4904e-04


In [20]:
# Print accuracy results
print("\nModel Performance Summary:")
print(f"Embedding + LSTM Model Accuracy: {lstm_eval[1]:.4f}")
print(f"Embedding + GRU + SimpleRNN Model Accuracy: {gru_rnn_eval[1]:.4f}")
print(f"Encoder-Decoder Model Accuracy: {encoder_decoder_eval[1]:.4f}")
print(f"Bidirectional LSTM Model Accuracy: {bilstm_eval[1]:.4f}")


Model Performance Summary:
Embedding + LSTM Model Accuracy: 1.0000
Embedding + GRU + SimpleRNN Model Accuracy: 1.0000
Encoder-Decoder Model Accuracy: 0.0001
Bidirectional LSTM Model Accuracy: 1.0000


In [21]:
# Generate predictions on sample data
print("\nGenerating sample predictions from Bidirectional LSTM Model:")
sample_input = tf.random.uniform((1, 50), minval=0, maxval=vocab_size, dtype=tf.int32)
prediction = bidirectional_lstm_model.predict(sample_input)
print("Sample Prediction Output:", prediction)


Generating sample predictions from Bidirectional LSTM Model:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 388ms/step
Sample Prediction Output: [[0.00467125]]
